### Combining data from OPUS for model training 

In [ ]:
"""

import pandas as pd

# Пути к файлам (замените на свои локальные пути)
ru_file = "TED2020.en-ru.ru"
en_file = "TED2020.en-ru.en"
output_file = "TED2020_translations.xlsx"

# Читаем файлы построчно
with open(ru_file, "r", encoding="utf-8") as f:
    ru_sentences = f.readlines()

with open(en_file, "r", encoding="utf-8") as f:
    en_sentences = f.readlines()

# Проверяем, что количество строк совпадает
if len(ru_sentences) != len(en_sentences):
    print("Ошибка: Файлы содержат разное количество строк!")
else:
    print(f"Загружено {len(ru_sentences)} пар предложений.")

# Создаём DataFrame
df = pd.DataFrame({"Russian": [s.strip() for s in ru_sentences], 
                   "English": [s.strip() for s in en_sentences]})

# Сохраняем в Excel
df.to_excel(output_file, index=False, engine="openpyxl")

print(f"🎉 Данные успешно сохранены в файл {output_file}")"

"""


### Data preparing 

In [ ]:
"""

import pandas as pd
from sklearn.model_selection import train_test_split

# Load dataset
df = pd.read_excel("TED2020_translations.xlsx")

# Split into train, validation, and test sets
train, temp = train_test_split(df, test_size=0.2, random_state=42)
val, test = train_test_split(temp, test_size=0.5, random_state=42)

# Save datasets as Excel files
train.to_excel("train.xlsx", index=False, engine="openpyxl")
val.to_excel("val.xlsx", index=False, engine="openpyxl")
test.to_excel("test.xlsx", index=False, engine="openpyxl")

print(f"Data prepared and saved: train({len(train)}), val({len(val)}), test({len(test)})")"

"""

Data prepared and saved: train(312011), val(39001), test(39002)


### Cleaning the data 

In [9]:
import pandas as pd

# Load training and validation data from Excel
train_data = pd.read_excel("train.xlsx", engine="openpyxl")
val_data = pd.read_excel("val.xlsx", engine="openpyxl")

# Drop NaN values and convert everything to strings
train_data = train_data.dropna().astype(str)
val_data = val_data.dropna().astype(str)

# Save cleaned data (optional step for debugging)
train_data.to_excel("train_cleaned.xlsx", index=False, engine="openpyxl")
val_data.to_excel("val_cleaned.xlsx", index=False, engine="openpyxl")

print("Data cleaned and saved as train_cleaned.xlsx & val_cleaned.xlsx")

Data cleaned and saved as train_cleaned.xlsx & val_cleaned.xlsx


### Train the MarianMT Model

In [2]:
!pip install transformers datasets torch sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 15.0 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 56.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 74.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 76.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.7 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 73.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 70.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 81.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 85.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 72.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 69.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
#!pip install transformers[torch] accelerate>=0.26.0

In [2]:
### Small dataset for fast training

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load dataset
df = pd.read_excel("train_cleaned.xlsx", engine="openpyxl")

# Take only a subset of data (e.g., 10,000 samples)
subset_size = 10000
df = df.sample(n=subset_size, random_state=42)  # Randomly select 10,000 rows

# Split into train/validation
train, val = train_test_split(df, test_size=0.2, random_state=42)

# Save the smaller dataset
train.to_excel("train_small.xlsx", index=False, engine="openpyxl")
val.to_excel("val_small.xlsx", index=False, engine="openpyxl")

print(f"✅ Prepared smaller dataset with {len(train)} training and {len(val)} validation samples.")

✅ Prepared smaller dataset with 8000 training and 2000 validation samples.


In [ ]:
### Second try training with small dataset

In [3]:
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import Dataset

# Load the smaller dataset
train_data = pd.read_excel("train_small.xlsx", engine="openpyxl")
val_data = pd.read_excel("val_small.xlsx", engine="openpyxl")

# Convert to Hugging Face Dataset format
train_dataset = Dataset.from_pandas(train_data)
val_dataset = Dataset.from_pandas(val_data)

# Load pre-trained MarianMT model
model_name = "Helsinki-NLP/opus-mt-ru-en"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# Tokenization function
def preprocess_function(examples):
    inputs = tokenizer(examples["Russian"], max_length=128, truncation=True, padding="max_length")
    targets = tokenizer(examples["English"], max_length=128, truncation=True, padding="max_length")
    inputs["labels"] = targets["input_ids"]
    return inputs

# Apply tokenization
train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)

# Define training arguments (optimized for faster training)
training_args = Seq2SeqTrainingArguments(
    output_dir="custom_translation_model",  # ✅ Save model in a clear directory
    num_train_epochs=2,  # ✅ Reduce epochs for faster training
    per_device_train_batch_size=16,  # ✅ Increase batch size to speed up training
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_steps=50,
    save_total_limit=1  # ✅ Keep only the latest checkpoint
)

# Initialize Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Start training
trainer.train()

# Save trained model in a clear directory
model.save_pretrained("custom_translation_model")
tokenizer.save_pretrained("custom_translation_model")

print("🎉 Training completed! Model saved in 'custom_translation_model'")


2025-03-18 19:44:58.328579: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-18 19:44:58.341621: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742327098.356305   18486 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742327098.360866   18486 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1742327098.373535   18486 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.795100,0.730192
2,0.691100,0.678546


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:2810: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 6, 'bad_words_ids': [[62517]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


🎉 Training completed! Model saved in 'custom_translation_model'


In [ ]:
### Second try, others hyperparametrs

In [2]:
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import Dataset

# Load the smaller dataset
train_data = pd.read_excel("train_small.xlsx", engine="openpyxl")
val_data = pd.read_excel("val_small.xlsx", engine="openpyxl")

# Convert to Hugging Face Dataset format
train_dataset = Dataset.from_pandas(train_data)
val_dataset = Dataset.from_pandas(val_data)

# Load pre-trained MarianMT model
model_name = "Helsinki-NLP/opus-mt-ru-en"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# Tokenization function
def preprocess_function(examples):
    inputs = tokenizer(examples["Russian"], max_length=128, truncation=True, padding="max_length")
    targets = tokenizer(examples["English"], max_length=128, truncation=True, padding="max_length")
    inputs["labels"] = targets["input_ids"]
    return inputs

# Apply tokenization
train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)

# Optimized Training Arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="custom_translation_model_v2",  # ✅ New folder for improved model
    num_train_epochs=4,  # ✅ More epochs for better training
    per_device_train_batch_size=32,  # ✅ Larger batch size (if enough memory)
    gradient_accumulation_steps=4,  # ✅ Effective larger batch size if memory is limited
    learning_rate=3e-5,  # ✅ Slower learning rate for stability
    weight_decay=0.01,  # ✅ Regularization to prevent overfitting
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_steps=50,
    save_total_limit=1
)

# Initialize Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Start training
trainer.train()

# Save trained model
model.save_pretrained("custom_translation_model_v2")
tokenizer.save_pretrained("custom_translation_model_v2")

print("Training completed! Model saved in 'custom_translation_model_v2'")

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,2.167600,1.126219
2,1.112500,0.946095
3,0.921500,0.869714


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:2810: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 6, 'bad_words_ids': [[62517]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


Training completed! Model saved in 'custom_translation_model_v2'


In [ ]:
### Third model

In [7]:
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import Dataset

# Load dataset
train_data = pd.read_excel("train_small.xlsx", engine="openpyxl")
val_data = pd.read_excel("val_small.xlsx", engine="openpyxl")

# Convert to Hugging Face Dataset format
train_dataset = Dataset.from_pandas(train_data)
val_dataset = Dataset.from_pandas(val_data)

# Load model and tokenizer
model_name = "Helsinki-NLP/opus-mt-ru-en"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# Preprocessing function with label smoothing support
def preprocess_function(examples):
    model_inputs = tokenizer(examples["Russian"], max_length=128, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["English"], max_length=128, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Tokenize datasets
train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)

# Training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="custom_translation_model_v1_5",
    num_train_epochs=3,
    per_device_train_batch_size=32,
    learning_rate=3e-5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_steps=50,
    save_total_limit=1,
    predict_with_generate=True  # Required for generation-based eval
)


# Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Train
trainer.train()

# Save model
model.save_pretrained("custom_translation_model_v1_5")
tokenizer.save_pretrained("custom_translation_model_v1_5")

print("✅ Improved model (v1.5) training complete and saved.")

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3970: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.320800,0.315128
2,0.291100,0.315201
3,0.277000,0.316235


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:2810: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 6, 'bad_words_ids': [[62517]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


✅ Improved model (v1.5) training complete and saved.


In [ ]:
### With EarlyStopping

In [9]:
import pandas as pd
from transformers import (
    MarianMTModel,
    MarianTokenizer,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    EarlyStoppingCallback
)
from datasets import Dataset

# 📌 Step 1: Load the dataset
train_data = pd.read_excel("train_small.xlsx", engine="openpyxl")
val_data = pd.read_excel("val_small.xlsx", engine="openpyxl")

train_dataset = Dataset.from_pandas(train_data)
val_dataset = Dataset.from_pandas(val_data)

# 📌 Step 2: Load model and tokenizer
model_name = "Helsinki-NLP/opus-mt-ru-en"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# 📌 Step 3: Preprocessing
def preprocess_function(examples):
    model_inputs = tokenizer(examples["Russian"], max_length=128, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["English"], max_length=128, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)

# 📌 Step 4: Training arguments with EarlyStopping setup
training_args = Seq2SeqTrainingArguments(
    output_dir="custom_translation_model_v1_5_earlystop",
    num_train_epochs=10,  # max limit, early stopping will stop earlier if needed
    per_device_train_batch_size=32,
    learning_rate=3e-5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_steps=50,
    save_total_limit=1,
    predict_with_generate=True,
    load_best_model_at_end=True,  # ⬅️ restores best model
    metric_for_best_model="eval_loss",  # based on validation loss
    greater_is_better=False
)

# 📌 Step 5: Trainer with EarlyStoppingCallback
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]  # ⏹️ stop after 2 non-improving epochs
)

# 📌 Step 6: Train and save
trainer.train()

model.save_pretrained("custom_translation_model_v1_5_earlystop")
tokenizer.save_pretrained("custom_translation_model_v1_5_earlystop")

print("✅ Training complete! Best model saved in 'custom_translation_model_v1_5_earlystop'")

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3970: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.321400,0.315831
2,0.289000,0.317355
3,0.266500,0.320317


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:2810: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 6, 'bad_words_ids': [[62517]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.encoder.embed_positions.weight', 'model.decoder.embed_tokens.weight', 'model.decoder.embed_positions.weight', 'lm_head.weight'].


✅ Training complete! Best model saved in 'custom_translation_model_v1_5_earlystop'


In [4]:
import pandas as pd
import torch
from transformers import MarianMTModel, MarianTokenizer

# Загружаем тестовый датасет
file_path = "assembly_WER.xlsx"
df = pd.read_excel(file_path, engine="openpyxl")

# Проверяем, есть ли колонка с текстом для перевода
if "Sentence" not in df.columns:
    raise ValueError("❌ В файле нет колонки 'Sentence'. Проверьте структуру файла!")

# Загружаем обе модели
model1_name = "custom_translation_model"
model2_name = "custom_translation_model_v2"

tokenizer1 = MarianTokenizer.from_pretrained(model1_name)
tokenizer2 = MarianTokenizer.from_pretrained(model2_name)

model1 = MarianMTModel.from_pretrained(model1_name)
model2 = MarianMTModel.from_pretrained(model2_name)

# Проверяем, есть ли GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
model1.to(device)
model2.to(device)

# Функция для пакетного перевода
def batch_translate(sentences, model, tokenizer):
    inputs = tokenizer(sentences, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)
    translated = model.generate(**inputs)
    return tokenizer.batch_decode(translated, skip_special_tokens=True)

# Разбиваем текст на batch-и по 16 предложений
batch_size = 16

df["Translation_Model_1"] = [
    trans for i in range(0, len(df), batch_size)
    for trans in batch_translate(df["Sentence"].astype(str).tolist()[i:i + batch_size], model1, tokenizer1)
]

df["Translation_Model_2"] = [
    trans for i in range(0, len(df), batch_size)
    for trans in batch_translate(df["Sentence"].astype(str).tolist()[i:i + batch_size], model2, tokenizer2)
]

# Сохраняем результат в новый Excel-файл
output_file = "translated_assembly_WER.xlsx"
df.to_excel(output_file, index=False, engine="openpyxl")

print(f"✅ Перевод завершён! Результаты сохранены в '{output_file}'")

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


✅ Перевод завершён! Результаты сохранены в 'translated_assembly_WER.xlsx'


In [2]:
import pandas as pd
import torch
from transformers import MarianMTModel, MarianTokenizer

# Загружаем тестовый датасет
file_path = "assembly_WER.xlsx"
df = pd.read_excel(file_path, engine="openpyxl")

# Проверяем, есть ли колонка с текстом для перевода
if "Sentence" not in df.columns:
    raise ValueError("❌ В файле нет колонки 'Sentence'. Проверьте структуру файла!")

# Загружаем обе модели
model1_name = "custom_translation_model"
model2_name = "custom_translation_model_v2"

tokenizer1 = MarianTokenizer.from_pretrained(model1_name)
tokenizer2 = MarianTokenizer.from_pretrained(model2_name)

model1 = MarianMTModel.from_pretrained(model1_name).to("cuda")  # ✅ Сразу на GPU
model2 = MarianMTModel.from_pretrained(model2_name).to("cuda")  # ✅ Сразу на GPU

# Функция для пакетного перевода
def batch_translate(sentences, model, tokenizer):
    inputs = tokenizer(sentences, return_tensors="pt", padding=True, truncation=True, max_length=128).to("cuda")
    translated = model.generate(**inputs)
    return tokenizer.batch_decode(translated, skip_special_tokens=True)

# Разбиваем текст на batch-и по 16 предложений
batch_size = 16

df["Translation_Model_1"] = [
    trans for i in range(0, len(df), batch_size)
    for trans in batch_translate(df["Sentence"].astype(str).tolist()[i:i + batch_size], model1, tokenizer1)
]

df["Translation_Model_2"] = [
    trans for i in range(0, len(df), batch_size)
    for trans in batch_translate(df["Sentence"].astype(str).tolist()[i:i + batch_size], model2, tokenizer2)
]

# Сохраняем результат в новый Excel-файл
output_file = "translated_assembly_WER.xlsx"
df.to_excel(output_file, index=False, engine="openpyxl")

print(f"✅ Перевод завершён! Результаты сохранены в '{output_file}'")

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


✅ Перевод завершён! Результаты сохранены в 'translated_assembly_WER.xlsx'


In [ ]:
### First try of training

In [1]:
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import Dataset

# Load cleaned training and validation data from Excel
train_data = pd.read_excel("train_cleaned.xlsx", engine="openpyxl")
val_data = pd.read_excel("val_cleaned.xlsx", engine="openpyxl")

# Convert Pandas DataFrame to Dataset format
train_dataset = Dataset.from_pandas(train_data)
val_dataset = Dataset.from_pandas(val_data)

# Load the MarianMT model and tokenizer
model_name = "Helsinki-NLP/opus-mt-ru-en"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# Preprocessing function for tokenization
def preprocess_function(examples):
    inputs = tokenizer(examples["Russian"], max_length=128, truncation=True, padding="max_length")
    targets = tokenizer(examples["English"], max_length=128, truncation=True, padding="max_length")
    inputs["labels"] = targets["input_ids"]
    return inputs

# Apply tokenization with batch processing
train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)

# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir=".",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2
)

# Initialize Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Start training
trainer.train()

# Save trained model
model.save_pretrained(".")
tokenizer.save_pretrained(".")

print("Training completed! Model saved")

2025-03-18 16:31:51.257754: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-18 16:31:51.270838: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742315511.285192   15960 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742315511.289643   15960 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1742315511.302109   15960 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

Map:   0%|          | 0/309030 [00:00<?, ? examples/s]

Map:   0%|          | 0/38613 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.443300,0.363478
2,0.367000,0.337283
3,0.293500,0.327943


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:2810: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 6, 'bad_words_ids': [[62517]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


Training completed! Model saved
